In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 🏗️ Investment Decisions: MPK vs. Cost of Capital

How do firms decide how much capital ($K$) to employ? A core principle is that firms adjust their capital stock to maximize profits. This involves comparing the benefit of adding one more unit of capital (its marginal product) with the cost of employing that unit.

This notebook visualizes this comparison using the **Marginal Product of Capital (MPK)** and the **user cost of capital**.

# ⚙️ The Optimal Capital Stock Condition

A profit-maximizing firm will invest in additional capital as long as the extra output generated by that capital (MPK) is greater than or equal to the cost of using that capital for one period.

1.  **Marginal Product of Capital (MPK):** The additional output produced by one extra unit of capital, holding other inputs constant. Assuming a Cobb-Douglas production function $Y = A K^{\alpha} L^{1-\alpha}$, and holding $A$ and $L$ constant for simplicity in this partial equilibrium view:
    $$MPK = \frac{\partial Y}{\partial K} = \alpha A K^{\alpha-1}$$
    Note that MPK is positive but diminishes as $K$ increases ($\alpha < 1$).

2.  **User Cost of Capital:** The total cost incurred by the firm for using one unit of capital for one period. In a simple model, it consists of:
    * The real interest rate ($r$), representing the opportunity cost of funds (either foregone interest income if using own funds, or borrowing cost).
    * The depreciation rate ($\delta$), representing the wear and tear or obsolescence of capital.
    $$ \text{User Cost} = r + \delta $$

**Optimality Condition:** The firm adjusts its capital stock until the benefit equals the cost:
$$MPK = r + \delta$$
$$\alpha A (K^*)^{\alpha-1} = r + \delta$$
This equation implicitly defines the firm's desired or **optimal capital stock ($K^*$**). If the firm's current $K$ is below $K^*$, it has an incentive to invest. If $K$ is above $K^*$, it may disinvest or let capital depreciate.

# 🧮 Tobin's q Theory

Tobin's q theory provides another perspective on investment decisions. It compares the market value of installed capital to its replacement cost. A simplified version often used in this context relates the *marginal* value of capital (MPK) to its cost:

$$ \text{Marginal } q = \frac{MPK}{r + \delta} $$

* **If $q > 1$:** The marginal product of capital exceeds its user cost ($MPK > r + \delta$). This means adding a unit of capital generates more value than it costs. **Incentive to Invest:** Increase $K$.
* **If $q < 1$:** The marginal product of capital is less than its user cost ($MPK < r + \delta$). Adding capital costs more than the value it generates. **Incentive to Disinvest:** Decrease $K$ (or don't replace depreciating capital).
* **If $q = 1$:** The marginal product equals the user cost ($MPK = r + \delta$). The firm is at its optimal capital stock ($K = K^*$). **No incentive to change investment** purely based on this condition.

The simulation below calculates MPK, the user cost, and this marginal $q$ for a given level of capital $K$, and plots the relationship.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def investment_decision_model(A=1.0, alpha=0.33, r=0.04, delta=0.06, K_current=20.0, K_max_plot=100):
    """
    Calculates MPK, user cost, Tobin's q, and plots the investment decision framework.

    Args:
        A (float): Total Factor Productivity (TFP).
        alpha (float): Capital share (output elasticity of capital).
        r (float): Real interest rate.
        delta (float): Depreciation rate.
        K_current (float): Current level of capital for evaluation.
        K_max_plot (float): Maximum capital level for the plot range.
    """
    # Input validation
    alpha = np.clip(alpha, 0.01, 0.99)
    r = max(r, 0)
    delta = max(delta, 0)
    A = max(A, 0.1)
    K_current = max(K_current, 0.1)
    K_max_plot = max(K_max_plot, K_current * 1.2, 10) # Ensure plot range is adequate

    # --- Calculations ---
    # User cost of capital
    user_cost = r + delta

    # MPK at the current capital level K_current
    # MPK = alpha * A * K^(alpha - 1)
    mpk_at_K_current = alpha * A * (K_current**(alpha - 1))

    # Simplified Marginal Tobin's q
    # Avoid division by zero if user_cost is zero
    if user_cost > 1e-9:
        q_marginal = mpk_at_K_current / user_cost
    else:
        q_marginal = np.inf if mpk_at_K_current > 0 else 0 # Or NaN

    # Optimal Capital Stock K* (where MPK = user_cost)
    # alpha * A * (K*)^(alpha - 1) = user_cost
    # (K*)^(alpha - 1) = user_cost / (alpha * A)
    # K* = (user_cost / (alpha * A))**(1 / (alpha - 1))
    optimal_K_star = np.nan # Default to NaN
    if user_cost > 0 and alpha * A > 0:
        base = user_cost / (alpha * A)
        exponent = 1.0 / (alpha - 1.0) # This exponent is negative
        if base > 0:
             try:
                 # Use log-exp trick for potentially large negative exponents
                 log_K_star = exponent * np.log(base)
                 # Add reasonable bounds if needed
                 # log_K_star = np.clip(log_K_star, -10, 10)
                 optimal_K_star = np.exp(log_K_star)
             except (ValueError, OverflowError):
                 optimal_K_star = np.nan # Handle potential math errors


    # --- Calculations for Plotting ---
    K_vals = np.linspace(1, K_max_plot, 200) # Capital range for plot
    MPK_vals = alpha * A * (K_vals**(alpha - 1))
    cost_line = np.full_like(K_vals, user_cost) # Horizontal line for user cost

    # --- Plotting ---
    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot MPK curve
    ax.plot(K_vals, MPK_vals, label='Marginal Product of Capital (MPK)', color='darkblue', linewidth=2.5)

    # Plot User Cost line
    ax.plot(K_vals, cost_line, color='firebrick', linestyle='--', linewidth=2, label=f'User Cost (r + δ = {user_cost:.3f})')

    # Mark Current K and corresponding MPK
    ax.axvline(K_current, color='gray', linestyle=':', linewidth=1.5, label=f'Current K = {K_current:.1f}')
    ax.scatter(K_current, mpk_at_K_current, color='black', s=80, zorder=5)
    ax.annotate(f'MPK = {mpk_at_K_current:.3f}\n(at Current K)', xy=(K_current, mpk_at_K_current),
                xytext=(15, 15), textcoords='offset points',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'),
                fontsize=9, bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.7))

    # Mark Optimal K* (if calculable and within plot range)
    if not np.isnan(optimal_K_star) and optimal_K_star <= K_max_plot and optimal_K_star > 0:
        ax.axvline(optimal_K_star, color='lime', linestyle='-', linewidth=2, alpha=0.7, label=f'Optimal K* ≈ {optimal_K_star:.1f}')
        ax.scatter(optimal_K_star, user_cost, color='lime', s=100, zorder=5, edgecolors='black')
        ax.annotate(f'Optimal K*\n(MPK = Cost)', xy=(optimal_K_star, user_cost),
                    xytext=(-30, -40) if optimal_K_star > K_max_plot*0.7 else (30, -40), textcoords='offset points',
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=-0.2', color='darkgreen'),
                    fontsize=9, bbox=dict(boxstyle='round,pad=0.3', fc='lightgreen', alpha=0.7))


    ax.set_title("Investment Decision: MPK vs. User Cost of Capital")
    ax.set_xlabel("Capital Stock (K)")
    ax.set_ylabel("Marginal Product / User Cost")
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.legend(loc='upper right')
    ax.set_xlim(0, K_max_plot)
    ax.set_ylim(bottom=0, top=max(MPK_vals.max(), user_cost) * 1.2) # Adjust ylim dynamically

    plt.tight_layout()
    plt.show()

    # --- Display Summary Results ---
    # Determine investment signal based on q
    if np.isclose(q_marginal, 1.0):
        invest_signal = "⚖️ Indifferent: q ≈ 1 (K ≈ K*)"
        invest_color = "orange"
    elif q_marginal > 1.0:
        invest_signal = f"✅ Invest: q ≈ {q_marginal:.2f} > 1 (MPK > Cost)"
        invest_color = "green"
    elif q_marginal < 1.0:
        invest_signal = f"❌ Do Not Invest: q ≈ {q_marginal:.2f} < 1 (MPK < Cost)"
        invest_color = "red"
    else: # Handle NaN case
        invest_signal = "⚠️ Cannot determine signal (q is undefined)"
        invest_color = "grey"


    results_md = f"""
    ### 📊 Investment Analysis at K = {K_current:.1f}:

    * **Marginal Product of Capital (MPK):** {mpk_at_K_current:.4f}
    * **User Cost of Capital (r + δ):** {user_cost:.4f} (Interest Rate: {r:.2%}, Depreciation: {delta:.2%})
    * **Marginal Tobin's q (MPK / Cost):** {q_marginal:.3f}
    * **Optimal Capital Stock (K\* where MPK=Cost):** {optimal_K_star:.2f}
    * **Investment Signal:** <font color='{invest_color}'>**{invest_signal}**</font>
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    investment_decision_model,
    A=FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='TFP (A):', style=style, layout=layout),
    alpha=FloatSlider(value=0.33, min=0.1, max=0.8, step=0.01, description='Capital Share (alpha α):', style=style, layout=layout, readout_format='.2f'),
    r=FloatSlider(value=0.04, min=0.0, max=0.15, step=0.005, description='Interest Rate (r):', style=style, layout=layout, readout_format='.1%'),
    delta=FloatSlider(value=0.06, min=0.0, max=0.15, step=0.005, description='Depreciation (delta δ):', style=style, layout=layout, readout_format='.1%'),
    K_current=FloatSlider(value=20.0, min=1, max=200, step=1, description='Current Capital (K):', style=style, layout=layout),
    K_max_plot=FloatSlider(value=100.0, min=20, max=500, step=10, description='Max K for Plot:', style=style, layout=layout)
);


<>:136: SyntaxWarning: invalid escape sequence '\*'
<>:136: SyntaxWarning: invalid escape sequence '\*'
/var/folders/kz/cc2y6j3953z1bgkfgj5xk9yc0000gn/T/ipykernel_87034/397937533.py:136: SyntaxWarning: invalid escape sequence '\*'
  display(Markdown(results_md))


interactive(children=(FloatSlider(value=1.0, description='TFP (A):', layout=Layout(width='95%'), max=3.0, min=…

# 🏗️ Investment Theory: MPK, Interest Rates, and Tobin’s q

Firms choose capital \( K \) to maximize profits.  
They invest if the return on capital exceeds its cost.

# 🔑 First-Order Condition:

\[
\text{Marginal Product of Capital (MPK)} = r + \delta
\]

- \( r \): real interest rate  
- \( \delta \): depreciation rate


# 💡 Intuition:
- If \( MPK > r + \delta \): **invest more**
- If \( MPK < r + \delta \): **invest less**


# 🧮 Tobin’s q Theory:

\[
q = \frac{\text{Value of Installed Capital}}{\text{Replacement Cost}}
\]

> If \( q > 1 \), the firm invests  
> If \( q < 1 \), it does not
